In [ ]:
%load_ext autoreload
%autoreload 2

from typing import Optional
import pandas as pd
import numpy as np
import pathlib
import seaborn as sns
from covidactnow.datapublic.common_fields import CommonFields
from libs.datasets import combined_datasets
from libs.datasets import AggregationLevel
from libs import top_level_metrics
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 3000

sns.set()

In [ ]:
us_timeseries = combined_datasets.load_us_timeseries_dataset()

In [ ]:
mass_timeseries = us_timeseries.get_subset(aggregation_level=AggregationLevel.STATE, state="MA")
# Get recent data
mass_timeseries.get_data(after="2020-08-01")
# equivalent to
# us_timeseries.get_data(aggregation_level=AggregationLevel.STATE, state="MA", after="2020-08-01")
# or
# mass_data = mass_timeseries.data
# mass_data.loc[mass_data[CommonFields.DATE] > "2020-08-01"]
# or
# mass_data.loc[mass_data['date'] > "2020-08-01"]


In [ ]:
top_level_metrics.calculate_top_level_metrics_for_fips("25")